## Visualizing Forest Development in India

#### By: Julia Mann and Luca Schmidt

In [ ]:
# imports
import pandas as pd
import numpy as np
import geopandas as gpd
import os
from scipy import stats

# plotting
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from tueplots import bundles


In [ ]:
# set path
path = '/Users/majuju/Desktop/Data Literacy/'

In [ ]:
# import forest data and keys
forest_data = pd.read_csv(path + 'shrug_vcf_wide.csv')
location_id = pd.read_csv(path + 'shrug_pc11_district_key.csv')

# import district level shapefiles
df1 = gpd.read_file( path + '2011_Dist.shp')

In [ ]:
# transforming district names to lowercase
df1['district'] = df1.DISTRICT.str.lower()

In [ ]:
# merge forest data and keys
df2 = pd.merge(location_id, forest_data, on=['shrid','shrid'], how='left')
df2.head()

### Visualizations

In [ ]:
# bar plot of percent forest per state
df_state = df2
df_state['id'] = df_state.index
df_state =  pd.wide_to_long(df_state, ['total_forest', 'max_forest'], i='id', j='year').reset_index()

In [ ]:
# sum total forest coverage by state in year 2019
df_state_2019 = df_state[df_state['year'] == 2019]
df_state_2019 = df_state_2019.groupby(['pc11_state_name'])[['total_forest','num_cells']].apply(sum).reset_index()
df_state_2019['percent'] = df_state_2019['total_forest']/df_state_2019['num_cells']

In [ ]:
plt.figure(figsize=(10,6))

# make barplot
with plt.rc_context(bundles.neurips2021(usetex=False,family='serif')):
    sns.barplot(x='pc11_state_name', 
            y='percent', 
            data=df_state_2019,
            palette='Greens_r',
            order=df_state_2019.sort_values('percent',ascending = False).pc11_state_name)

# set labels
plt.xlabel("State")
plt.ylabel("% Forest Coverage")
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('forest_cover_states.pdf', bbox_inches='tight')

In [ ]:
# sum total forest in year 2000
df_state_2000 = df_state[df_state['year'] == 2000]
df_state_2000 = df_state_2000.groupby(['pc11_state_name'])[['total_forest','num_cells']].apply(sum).reset_index()
df_state_2000['percent'] = df_state_2000['total_forest']/df_state_2000['num_cells']

# calculate percent change
percent_change = ((df_state_2019['percent']-df_state_2000['percent'])/df_state_2000['percent'])*100
state_names = df_state_2019['pc11_state_name']
state_names = pd.DataFrame(state_names)
df_change = state_names.join(percent_change)

In [ ]:
for i in range(35):
    # color of bar chart is set to red if the sales 
    # is < 60000 and green otherwise
    df_change['colors'] = ['red' if float(
        x) < 0 else 'green' for x in df_change['percent']]
  
 # sort values from lowest to highest
df_change.sort_values('percent', inplace=True)
  
# reset initial index in Dataframe to None
df_change.reset_index(inplace=True)


# draw plot
plt.figure(figsize=(10,6))
  
# plotting the lines
with plt.rc_context(bundles.neurips2021(usetex=False,family='serif')):
    plt.vlines(x=df_change.index, ymin=0, ymax=df_change.percent,color=df_change.colors, alpha=0.4, linewidth=5)
  
 # Setting the labels of x-axis and y-axis
plt.gca().set(ylabel='% Change Forest Coverage', xlabel='State')
  
# Setting Date to y-axis
plt.xticks(df_change.index, df_change.pc11_state_name, rotation = 90)

plt.savefig('percent_change_forest_states.pdf', bbox_inches='tight')

In [ ]:
# linear model
df_reg = df2
df_reg = df2.groupby(by=['pc11_district_name', 'pc11_district_id']).sum().reset_index()

# grouping by year
df_reg['id'] = df_reg.index
df_reg = pd.wide_to_long(df_reg, ['total_forest', 'max_forest'], i='id', j='year')
df_reg = df_reg.reset_index()
df_reg['avg_forest'] = df_reg['total_forest']/df_reg['num_cells']

df_reg = df_reg.groupby(['year'])[['total_forest', 'max_forest','avg_forest']].apply(sum).reset_index()
df_reg['avg_forest'] = df_reg['avg_forest']/622

In [ ]:
# make barplot
plt.figure(figsize=(8,5))

with plt.rc_context(bundles.neurips2021(usetex=False,family='serif')):
    sns.barplot(x='year', y="avg_forest", data=df_reg, color='Green')

# set labels
plt.xlabel("Year")
plt.ylabel("% Forest Coverage")
plt.xticks(rotation=45)
plt.tight_layout()

plt.savefig('percent_forest_per_year.pdf', bbox_inches='tight')

In [ ]:
x = df_reg['year']
y = df_reg['avg_forest']
res = stats.linregress(x, y)

# linear plot
plt.figure(figsize=(10,4))

with plt.rc_context(bundles.neurips2021(usetex=False,family='serif')):
    plt.plot(x, y,'o', label='original data', color='Green')
    plt.plot(x, res.intercept + res.slope*x, 'black', label='fitted line')

plt.xlabel('Year')
plt.ylabel('Forest Coverage')
ticks = [y for y in range(2000,2020)]
plt.xticks(df_reg['year'], ticks, rotation = 40)

plt.legend()
plt.show()

plt.savefig('forest_cover_trend.pdf', bbox_inches='tight')

## Choropleth

In [ ]:
# convert wide to long format
df2['id'] = df2.index
df2 = pd.wide_to_long(df2, ['total_forest', 'max_forest'], i='id', j='year')
df2 = df2.reset_index()

In [ ]:
from datetime import datetime, timedelta
import geopandas as gpd
import json
import plotly.express as px
from dateutil.relativedelta import relativedelta


#reading the shape file 
map_df = gpd.read_file('/Users/majuju/Downloads/Census_2011/2011_Dist.shp')
#Export it as GeoJSON
map_df.to_file("json_files\\India_Districts.json", driver='GeoJSON')

In [ ]:
with open('json_files\\India_Districts.json') as f:
      India_districts = json.load(f)

In [ ]:
#Check the keys of the json
print(India_districts["features"][0].keys())
print(India_districts["features"][0]['properties'].keys())

In [ ]:
India_districts["features"][0]['geometry']['coordinates'][0][0]

In [ ]:
#Round off the locations to 2 decimal places (about 1.1 km accuracy)
for i in range(0, len(India_districts["features"])):
    for j in range(0,len(India_districts["features"][i]['geometry']['coordinates'])):
        try:
            India_districts["features"][i]['geometry']['coordinates'][j] = np.round(np.array(India_districts["features"][i]['geometry']['coordinates'][j]),2)
        except:
            print(i,j)
            
#rounded off location data           
#print(India_districts["features"][0]['geometry']['coordinates'][0][0])
#print(India_districts["features"][0])

In [ ]:
India_districts["features"][0]['geometry']['coordinates'][0][0]

In [ ]:
# convert year to datetime
df2['year'] = pd.to_datetime(df2.year, format='%Y')


In [ ]:
df_d = map_df[['DISTRICT']]


In [ ]:
df_d['DISTRICT'] = df_d.DISTRICT.str.lower()


# dic of unmatched districts 
dic = {'bauda':'baudh', 'chamrajnagar':'chamarajanagar', 'dadra nagar haveli':'dadra nagar haveli', 'garhchiroli': 'gadchiroli', 'janjgir-champa': 'janjgir champa',  'kaimur (bhabua)': 'kaimur bhabua', 'kansiram nagar': 'kanshiram nagar', 'lahul & spiti':'lahul spiti',  'lawangtlai': 'lawngtlai', 
 'leh (ladakh)': 'leh ladakh', 'maharajganj': 'mahrajganj', 'marigaon': 'morigaon', 'nagappattinam': 'nagapattinam',  'nicobar': 'nicobars','north & middle andaman': 'north middle andaman',  'north 24 parganas': 'north twenty four parganas', 'pashchim medinipur': 'paschim medinipur',  'ri bhoi': ' ribhoi', 
'sant ravi das nagar(bhadohi)': 'sant ravidas nagar bhadohi', 'saraikela-kharsawan': 'saraikela kharsawan', 'saran (chhapra)': 'saran', 'siddharth nagar' :  'siddharthnagar','south 24 parganas': 'south twenty four parganas', 'virudunagar' : ' virudhunagar'
, 'west': 'west district', 'east': 'east district', 'north': 'north district', 'south': 'south district'}

# assimilate district names
for key in dic.keys():
    df_d['DISTRICT'] = df_d['DISTRICT'].replace(key, dic[key])
    


In [ ]:
# smallest year
date_min = df2['year'].min()

# create dataframe for district, variable of interest and year
date_min = df2['year'].min()
df_final = pd.DataFrame(columns=['DISTRICT','forest','date'])

# create subset of df for each year
for i in range(0,20):
    date = date_min + relativedelta(years=i)
    df_c = df2[df2['year'] == date]
    
    
    # temporary df to store the values
    df_c = df_c.groupby(['pc11_district_name'])['total_forest', 'num_cells'].sum().reset_index()
    df_c['forest'] = df_c['total_forest']/df_c['num_cells']
    
    df_t = df_c[['pc11_district_name', 'forest']]
    merged = df_d.set_index('DISTRICT').join(df_t.set_index('pc11_district_name'))
    
    merged['forest'].fillna(0,inplace=True)
    merged = merged.reset_index()
 
    merged['date'] = date
    merged['DISTRICT']= merged['index']
    merged = merged.drop('index',1)
    
    df_final = pd.concat([df_final, merged], ignore_index = True)
    
df_final['dt_str'] = df_final['date'].apply(lambda x: x.strftime("%Y"))


In [ ]:
# capitalize for matching purposes
df_final['DISTRICT'] = df_final.DISTRICT.str.capitalize()


In [ ]:

max_count = df_final['forest'].max()
fig = px.choropleth_mapbox(df_final, geojson = India_districts,
                      locations='DISTRICT', 
                      color= df_final['forest'],
                      color_continuous_scale="Greens",
                      range_color=(0, max_count),
                      featureidkey="properties.DISTRICT",
                      mapbox_style="carto-positron",
                      opacity = 0.5,
                      center = {"lat": 26.8467, "lon": 80.9462}, 
                      zoom = 4,
                      animation_frame='dt_str')

fig.update_geos(fitbounds="locations",visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},)

fig.show()




In [ ]:
fig.write_html('html_files\\plotly_mapbox_choro.html')